In [7]:
# FINAL RSM PIPELINE with REFINEMENT + R2-vs-Iteration PLOT
# Requirements:
# pip install pandas numpy pyDOE2 statsmodels seaborn matplotlib sympy scikit-learn openpyxl

import os
import pandas as pd
import numpy as np
from pyDOE2 import ccdesign
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
import sympy as sp
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ------------------------
# USER CONFIG
# ------------------------
DATA_PATH = "dataset5.xlsx"   # <- change to your file
SHEET_NAME = 0
OUT_DIR = "ccd_full_analysis_refined_with_plots"
os.makedirs(OUT_DIR, exist_ok=True)

# ------------------------
# 1) LOAD DATA
# ------------------------
df = pd.read_excel(DATA_PATH, sheet_name=SHEET_NAME)
df.columns = df.columns.str.strip()

required_cols = ['Strength (MPa)', 'BFC (%)', 'Temperature (°C)', 'Duration (min)', 'Batch type']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# ensure numeric
for c in ['Strength (MPa)', 'BFC (%)', 'Temperature (°C)', 'Duration (min)']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

print(f"\n✅ Loaded dataset with {len(df)} rows")
print(df[['BFC (%)','Temperature (°C)','Duration (min)']].describe().T[['min','max']])

# ------------------------
# 2) FACE-CENTERED CCD (α = 1) generation (uses observed ranges)
# ------------------------
factors = ['BFC (%)', 'Temperature (°C)', 'Duration (min)']
ranges = {f: (df[f].min(), df[f].max()) for f in factors}

k = 3
factorial_runs = 2 ** k
axial_runs = 2 * k
target_total = 20
remaining = target_total - (factorial_runs + axial_runs)
center_factorial = max(1, remaining // 2)
center_axial = max(1, remaining - center_factorial)

ccd_face = ccdesign(k, center=(center_factorial, center_axial), face='ccf')  # face-centered
ccd_df = pd.DataFrame(ccd_face, columns=['A_coded','B_coded','C_coded'])
for i, f in enumerate(factors):
    low, high = ranges[f]
    center = (low + high) / 2.0
    half = (high - low) / 2.0
    ccd_df[f] = center + half * ccd_df.iloc[:, i]
ccd_df['Run'] = np.arange(1, len(ccd_df) + 1)
ccd_df.to_excel(os.path.join(OUT_DIR, "FaceCentered_CCD_20runs.xlsx"), index=False)

print("\n=== Face-Centered CCD matrix (α=1) ===")
print(ccd_df.round(3))

# ------------------------
# 3) Descriptive statistics function (and save)
# ------------------------
def compute_and_save_descriptive(df, out_dir=OUT_DIR):
    numeric = df.select_dtypes(include=[np.number]).columns.tolist()
    desc = df[numeric].describe().T
    desc['CV%'] = (desc['std'] / desc['mean']) * 100
    desc = desc[['count','mean','std','min','25%','50%','75%','max','CV%']]
    desc.to_excel(os.path.join(out_dir, "Descriptive_Stats.xlsx"))
    print("\n=== Descriptive Statistics ===")
    print(desc.round(4))
    return desc

desc = compute_and_save_descriptive(df)

# ------------------------
# 4) split by test type
# ------------------------
df['TestType'] = df['Batch type'].apply(lambda x: 'Compressive' if 'COMPRESS' in str(x).upper()
                                        else ('Flexural' if 'FLEX' in str(x).upper() else 'Other'))

comp_df = df[df['TestType']=='Compressive'].copy()
flex_df = df[df['TestType']=='Flexural'].copy()
print(f"\nCompressive rows: {len(comp_df)} | Flexural rows: {len(flex_df)}")

# ------------------------
# 5) build X helper
# ------------------------
def build_X(df_in, factors_list):
    df_in = df_in.copy()
    n = len(df_in)
    X = pd.DataFrame({'const': np.ones(n)}, index=df_in.index)
    # main effects
    for f in factors_list:
        X[f] = df_in[f]
    # interactions
    for i in range(len(factors_list)):
        for j in range(i+1, len(factors_list)):
            X[f"{factors_list[i]}*{factors_list[j]}"] = df_in[factors_list[i]] * df_in[factors_list[j]]
    # quadratic terms
    for f in factors_list:
        X[f"{f}^2"] = df_in[f] ** 2
    return X

# ------------------------
# 6) fit full RSM (returns model and cleaned df)
# ------------------------
def fit_full_rsm(df_in, response='Strength (MPa)', factors_list=factors, label='Model'):
    print(f"\n--- Fitting full quadratic RSM for {label} ---")
    df_clean = df_in.replace([np.inf, -np.inf], np.nan).dropna(subset=[response] + factors_list).copy()
    if df_clean.shape[0] < 10:
        print(f"⚠️ Not enough complete rows for {label} (need >=10). Found: {df_clean.shape[0]}")
        return None, None
    X = build_X(df_clean, factors_list)
    y = df_clean[response]
    model = sm.OLS(y, X).fit()
    print(model.summary())
    return model, df_clean

comp_model, comp_clean = fit_full_rsm(comp_df, label='Compressive')
flex_model, flex_clean = fit_full_rsm(flex_df, label='Flexural')

# ------------------------
# 7) model refinement (backward elimination) with iteration history and R2 plot
# ------------------------
def refine_model_backward(df_clean, response, factors_list, label, p_cutoff=0.05, save_history=True):
    """
    Iteratively remove the least significant term (highest p-value > p_cutoff)
    following hierarchy: interactions → quadratics → main factors.
    """
    if df_clean is None:
        print(f"⚠️ No data to refine for {label}")
        return None, None, None

    X_full = build_X(df_clean, factors_list)
    y = df_clean[response]
    current_terms = list(X_full.columns)
    history = []
    iteration = 1

    print(f"\n🔁 Starting hierarchical backward elimination for {label}")
    print(f"Initial terms ({len(current_terms)}): {current_terms}")

    # Helper classification
    def term_type(term):
        if any('*' in term or ':' in term for _ in [term]):
            return 'interaction'
        elif '^2' in term:
            return 'quadratic'
        elif any(f in term for f in factors_list):
            return 'main'
        else:
            return 'other'

    while True:
        model = sm.OLS(y, X_full[current_terms]).fit()
        pvals = model.pvalues.drop('const', errors='ignore')

        # classify terms
        term_groups = {
            'interaction': {t: p for t, p in pvals.items() if '*' in t or ':' in t},
            'quadratic': {t: p for t, p in pvals.items() if '^2' in t},
            'main': {t: p for t, p in pvals.items() if t in factors_list},
        }

        # choose elimination group: interaction first, then quadratic, then main
        target_group = None
        for g in ['interaction', 'quadratic', 'main']:
            if any(p > p_cutoff for p in term_groups[g].values()):
                target_group = g
                break

        if target_group is None:
            print("✅ All remaining terms are significant (p <= {:.3f}). Stopping.".format(p_cutoff))
            break

        # find worst (highest p) in selected group
        group_pvals = term_groups[target_group]
        worst_term = max(group_pvals, key=group_pvals.get)
        max_p = group_pvals[worst_term]

        print(f"\n--- Iteration {iteration} ({target_group.upper()} removal) ---")
        print(model.summary())
        print(f"⚠️ Removing '{worst_term}' ({target_group}, p={max_p:.4f})")

        history.append({
            'Iteration': iteration,
            'Group': target_group,
            'RemovedTerm': worst_term,
            'PValue': max_p,
            'R2': model.rsquared,
            'AdjR2': model.rsquared_adj,
            'AIC': model.aic,
            'BIC': model.bic
        })

        current_terms.remove(worst_term)
        iteration += 1

        if len(current_terms) <= len(factors_list) + 1:  # keep constant + main factors minimum
            print("⚠️ Stopping: reached base model (main effects only).")
            break

    # Final model
    final_model = sm.OLS(y, X_full[current_terms]).fit()
    print("\n=== Final Refined Model Summary ===")
    print(final_model.summary())

    hist_df = pd.DataFrame(history)
    if save_history and not hist_df.empty:
        path = os.path.join(OUT_DIR, f"{label}_HierarchicalRefinementHistory.xlsx")
        hist_df.to_excel(path, index=False)
        print(f"📘 Saved refinement history to: {path}")

    return final_model, hist_df, current_terms


comp_refined, comp_hist, comp_terms = refine_model_backward(comp_clean, 'Strength (MPa)', factors, 'Compressive', p_cutoff=0.05)
flex_refined, flex_hist, flex_terms = refine_model_backward(flex_clean, 'Strength (MPa)', factors, 'Flexural', p_cutoff=0.05)

from scipy.optimize import minimize
from scipy import stats
import numpy as np
import pandas as pd
import os

def compute_anova_and_optimum(model, df_clean, label, factors, out_dir=OUT_DIR):
    """
    Computes a manual ANOVA table (since design_info missing for sm.OLS)
    and finds the optimum factor combination that maximizes predicted Strength (MPa).
    """
    if model is None or df_clean is None:
        print(f"⚠️ Missing data or model for {label}")
        return None, None

    print(f"\n=== ANOVA and Optimization for {label} ===")

    # ==================================================
    # 1️⃣ MANUAL ANOVA COMPUTATION (for sm.OLS models)
    # ==================================================
    try:
        ssr = np.sum((model.fittedvalues - np.mean(model.model.endog))**2)   # regression SS
        sse = np.sum(model.resid**2)                                        # residual SS
        sst = ssr + sse                                                     # total SS

        df_model = model.df_model
        df_resid = model.df_resid
        msr = ssr / df_model
        mse = sse / df_resid
        f_value = msr / mse
        p_value = 1 - stats.f.cdf(f_value, df_model, df_resid)  # ✅ FIXED: from scipy.stats

        anova_manual = pd.DataFrame({
            'Source': ['Regression', 'Residual', 'Total'],
            'DF': [df_model, df_resid, df_model + df_resid],
            'Adj SS': [ssr, sse, sst],
            'Adj MS': [msr, mse, np.nan],
            'F-Value': [f_value, np.nan, np.nan],
            'P-Value': [p_value, np.nan, np.nan]
        }).round(4)

        # Compute percentage contribution of regression
        anova_manual.loc[0, 'Contribution (%)'] = (ssr / sst) * 100
        anova_manual.loc[1, 'Contribution (%)'] = (sse / sst) * 100
        anova_manual.loc[2, 'Contribution (%)'] = np.nan

        print("\n--- ANOVA (Manual Computation) ---")
        print(anova_manual)
        anova_path = os.path.join(out_dir, f"{label}_ANOVA_manual.xlsx")
        anova_manual.to_excel(anova_path, index=False)
        print(f"📘 Saved ANOVA to: {anova_path}")

    except Exception as e:
        print(f"⚠️ Manual ANOVA failed for {label}: {e}")
        anova_manual = None

    # ==================================================
    # 2️⃣ OPTIMIZATION (maximize predicted response)
    # ==================================================
    print("\n--- Optimization of Predicted Strength ---")

    # Factor ranges
    bounds = [(df_clean[f].min(), df_clean[f].max()) for f in factors]

    # Define model prediction function
    def predict_strength(x):
        """
        x = [BFC, Temp, Duration]
        returns -predicted_strength (for minimization)
        """
        vals = dict(zip(factors, x))
        data = {c: [1] for c in model.params.index}  # initialize with const
        for name in model.params.index:
            if name == 'const':
                data[name] = [1]
            elif name in vals:
                data[name] = [vals[name]]
            elif '*' in name:  # interaction term
                terms = name.split('*')
                if all(t in vals for t in terms):
                    data[name] = [np.prod([vals[t] for t in terms])]
            elif '^2' in name:  # quadratic term
                base = name.split('^')[0].strip()
                if base in vals:
                    data[name] = [vals[base]**2]
            else:
                data[name] = [0]
        df_pred = pd.DataFrame(data)
        return -model.predict(df_pred)[0]  # negative for maximization

    # Perform bounded optimization
    result = minimize(
        predict_strength,
        x0=[df_clean[f].mean() for f in factors],
        bounds=bounds,
        method='L-BFGS-B'
    )

    opt_x = result.x
    max_pred = -result.fun

    opt_summary = pd.DataFrame({
        'Factor': factors + ['Predicted Strength (MPa)'],
        'Optimum Value': list(opt_x) + [max_pred]
    })
    opt_path = os.path.join(out_dir, f"{label}_OptimumParameters.xlsx")
    opt_summary.to_excel(opt_path, index=False)

    print(f"\nPredicted optimum for {label}:")
    for f, val in zip(factors, opt_x):
        print(f" → {f}: {val:.4f}")
    print(f" → Predicted Strength (MPa): {max_pred:.4f}")
    print(f"📘 Saved optimum parameters to: {opt_path}")

    return anova_manual, opt_summary



# ------------------------
# 8) Plot R2 and AdjR2 vs iteration for each refinement (if history exists)
# ------------------------
def plot_refinement_progress(hist_df, label):
    if hist_df is None or hist_df.empty:
        print(f"No refinement history for {label}")
        return
    plt.figure(figsize=(6,4))
    plt.plot(hist_df['Iteration'], hist_df['R2'], marker='o', label='R2')
    plt.plot(hist_df['Iteration'], hist_df['AdjR2'], marker='s', label='Adj R2')
    plt.xlabel('Iteration')
    plt.ylabel('Value')
    plt.title(f"{label} - R² and Adj R² vs Iteration")
    plt.grid(alpha=0.3)
    plt.legend()
    plt.tight_layout()
    path = os.path.join(OUT_DIR, f"{label}_R2_vs_Iteration.png")
    plt.savefig(path, dpi=300)
    plt.close()
    print(f"Saved R2 vs iteration plot: {path}")

plot_refinement_progress(comp_hist, 'Compressive')
plot_refinement_progress(flex_hist, 'Flexural')

# ------------------------
# 9) Diagnostics + equation export + VIF + metrics for final models
# ------------------------
def diagnostics_and_export(final_model, df_clean, label):
    if final_model is None:
        print(f"No final model for {label}. Skipping diagnostics.")
        return

    # residuals and fitted
    resid = final_model.resid
    fitted = final_model.fittedvalues

    # Residuals vs Fitted
    plt.figure(figsize=(6,4))
    plt.scatter(fitted, resid, alpha=0.7)
    plt.axhline(0, color='k', linestyle='--')
    plt.xlabel('Fitted')
    plt.ylabel('Residuals')
    plt.title(f'{label} - Residuals vs Fitted')
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{label}_Residuals_vs_Fitted_final.png"), dpi=300)
    plt.close()

    # QQ plot
    plt.figure(figsize=(6,4))
    sm.qqplot(resid, line='45', fit=True)
    plt.title(f'{label} - QQ Plot (final model)')
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{label}_QQ_final.png"), dpi=300)
    plt.close()

    # VIF (only for terms present)
    # build a design matrix for VIF (drop const)
    exog = final_model.model.exog
    names = final_model.model.exog_names
    design_df = pd.DataFrame(exog, columns=names)
    if 'const' in design_df.columns:
        design_no_const = design_df.drop(columns=['const'])
    elif 'Intercept' in design_df.columns:
        design_no_const = design_df.drop(columns=['Intercept'])
    else:
        design_no_const = design_df.copy()

    vif_list = []
    for i, col in enumerate(design_no_const.columns):
        try:
            vif_val = variance_inflation_factor(design_no_const.values, i)
        except Exception:
            vif_val = np.nan
        vif_list.append({'Term': col, 'VIF': vif_val})
    vif_df = pd.DataFrame(vif_list)
    vif_df.to_excel(os.path.join(OUT_DIR, f"{label}_VIF_final.xlsx"), index=False)

    # Save summary txt
    with open(os.path.join(OUT_DIR, f"{label}_FinalModel_Summary.txt"), "w") as f:
        f.write(final_model.summary().as_text())

    # Metrics
    y = df_clean['Strength (MPa)']
    y_pred = final_model.fittedvalues
    r2 = r2_score(y, y_pred)
    rmse = mean_squared_error(y, y_pred, squared=False)
    mae = mean_absolute_error(y, y_pred)

    metrics = {'R2': r2, 'RMSE': rmse, 'MAE': mae}
    pd.DataFrame([metrics]).to_excel(os.path.join(OUT_DIR, f"{label}_FinalMetrics.xlsx"), index=False)
    print(f"\n{label} final metrics: R2={r2:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}")

    # Export symbolic equation (map column names to A,B,C where appropriate)
    coeffs = final_model.params.to_dict()
    A,B,C = sp.symbols('A B C')  # order corresponds to factors list
    eq = coeffs.get('const', 0)
    # mapping: check presence of factor names in coeff keys
    for k, v in coeffs.items():
        if k == 'const':
            continue
        # linear
        if k == factors[0]:
            eq += v * A
        elif k == factors[1]:
            eq += v * B
        elif k == factors[2]:
            eq += v * C
        # interactions
        elif f"{factors[0]}*{factors[1]}" in k or ':' in k and factors[0] in k and factors[1] in k:
            eq += v * A * B
        elif f"{factors[0]}*{factors[2]}" in k or ':' in k and factors[0] in k and factors[2] in k:
            eq += v * A * C
        elif f"{factors[1]}*{factors[2]}" in k or ':' in k and factors[1] in k and factors[2] in k:
            eq += v * B * C
        # squares
        elif f"{factors[0]}^2" in k or f"I({factors[0]}" in k or f"{factors[0]}**2" in k:
            eq += v * A**2
        elif f"{factors[1]}^2" in k or f"I({factors[1]}" in k or f"{factors[1]}**2" in k:
            eq += v * B**2
        elif f"{factors[2]}^2" in k or f"I({factors[2]}" in k or f"{factors[2]}**2" in k:
            eq += v * C**2
        else:
            # fallback: append as textual term (rare)
            pass

    eq_s = sp.expand(eq)
    with open(os.path.join(OUT_DIR, f"{label}_Equation.txt"), "w") as f:
        f.write(f"{label} symbolic equation (A={factors[0]}, B={factors[1]}, C={factors[2]}):\n")
        f.write(sp.pretty(eq_s, use_unicode=True))

    print(f"Saved diagnostics, VIF, equation and metrics for {label} to {OUT_DIR}")

    return metrics, vif_df

comp_metrics, comp_vif = diagnostics_and_export(comp_refined, comp_clean, 'Compressive')
flex_metrics, flex_vif = diagnostics_and_export(flex_refined, flex_clean, 'Flexural')

comp_anova, comp_opt = compute_anova_and_optimum(comp_refined, comp_clean, "Compressive", factors)
flex_anova, flex_opt = compute_anova_and_optimum(flex_refined, flex_clean, "Flexural", factors)


# ------------------------
# 10) Produce final RSM 3D surface + contour plots (fixed 3rd factor at mean)
# ------------------------
def plot_surface_and_contour_from_model(model, df_clean, label):
    if model is None or df_clean is None:
        return
    f1, f2, f3 = factors
    grid_n = 60
    F1, F2 = np.meshgrid(np.linspace(df_clean[f1].min(), df_clean[f1].max(), grid_n),
                         np.linspace(df_clean[f2].min(), df_clean[f2].max(), grid_n))
    f3_val = df_clean[f3].mean()

    # prepare DF matching model terms
    DF = pd.DataFrame({
        'const': np.ones(F1.ravel().shape[0]),
        f1: F1.ravel(),
        f2: F2.ravel(),
        f3: np.full(F1.ravel().shape[0], f3_val),
        f"{f1}*{f2}": (F1 * F2).ravel(),
        f"{f1}*{f3}": (F1 * f3_val).ravel(),
        f"{f2}*{f3}": (F2 * f3_val).ravel(),
        f"{f1}^2": (F1**2).ravel(),
        f"{f2}^2": (F2**2).ravel(),
        f"{f3}^2": (f3_val**2) * np.ones(F1.ravel().shape[0])
    })
    # keep only the columns the model expects
    DF = DF[[c for c in DF.columns if c in model.params.index]]
    preds = model.predict(DF)
    Z = preds.values.reshape(F1.shape)

    # find optimum
    idx = np.unravel_index(np.nanargmax(Z), Z.shape)
    opt_x, opt_y, opt_z = F1[idx], F2[idx], Z[idx]

    # 3D plot
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(F1, F2, Z, cmap=cm.viridis, alpha=0.85)
    ax.scatter(opt_x, opt_y, opt_z, color='red', s=80, label=f'Optimum: {opt_z:.2f}')
    ax.set_xlabel(f1); ax.set_ylabel(f2); ax.set_zlabel('Predicted Strength (MPa)')
    ax.set_title(f"{label} - Refined RSM Surface (fixed {f3}={f3_val:.1f})")
    ax.legend()
    fig.colorbar(surf, shrink=0.5, aspect=10)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{label}_RefinedRSM_3D.png"), dpi=300)
    plt.close()

    # Contour
    plt.figure(figsize=(7,6))
    cp = plt.contourf(F1, F2, Z, levels=40, cmap='viridis')
    plt.scatter(opt_x, opt_y, color='red', s=50, label=f'Optimum: {opt_z:.2f}')
    plt.colorbar(cp, label='Predicted Strength (MPa)')
    plt.xlabel(f1); plt.ylabel(f2)
    plt.title(f"{label} - Refined RSM Contour (fixed {f3}={f3_val:.1f})")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{label}_RefinedRSM_Contour.png"), dpi=300)
    plt.close()

    print(f"Saved refined RSM 3D and contour plots for {label}. Optimum at ({f1}={opt_x:.3f}, {f2}={opt_y:.1f}) -> {opt_z:.3f} MPa")

plot_surface_and_contour_from_model(comp_refined, comp_clean, 'Compressive')
plot_surface_and_contour_from_model(flex_refined, flex_clean, 'Flexural')

print("\n🎯 RSM refinement pipeline complete. Outputs saved to:", OUT_DIR)



✅ Loaded dataset with 80 rows
                    min     max
BFC (%)             0.0     1.0
Temperature (°C)  200.0  1000.0
Duration (min)     20.0    60.0

=== Face-Centered CCD matrix (α=1) ===
    A_coded  B_coded  C_coded  BFC (%)  Temperature (°C)  Duration (min)  Run
0      -1.0     -1.0     -1.0      0.0             200.0            20.0    1
1       1.0     -1.0     -1.0      1.0             200.0            20.0    2
2      -1.0      1.0     -1.0      0.0            1000.0            20.0    3
3       1.0      1.0     -1.0      1.0            1000.0            20.0    4
4      -1.0     -1.0      1.0      0.0             200.0            60.0    5
5       1.0     -1.0      1.0      1.0             200.0            60.0    6
6      -1.0      1.0      1.0      0.0            1000.0            60.0    7
7       1.0      1.0      1.0      1.0            1000.0            60.0    8
8       0.0      0.0      0.0      0.5             600.0            40.0    9
9       0.0      0.0 

/Users/freshliannes.rosal/anaconda3/envs/Numerical/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



Compressive final metrics: R2=0.9615, RMSE=2.7092, MAE=2.1086
Saved diagnostics, VIF, equation and metrics for Compressive to ccd_full_analysis_refined_with_plots


/Users/freshliannes.rosal/anaconda3/envs/Numerical/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



Flexural final metrics: R2=0.7386, RMSE=0.3347, MAE=0.2264
Saved diagnostics, VIF, equation and metrics for Flexural to ccd_full_analysis_refined_with_plots

=== ANOVA and Optimization for Compressive ===

--- ANOVA (Manual Computation) ---
       Source    DF     Adj SS     Adj MS   F-Value  P-Value  Contribution (%)
0  Regression   5.0  7322.5876  1464.5175  169.6015      0.0         96.145158
1    Residual  34.0   293.5916     8.6350       NaN      NaN          3.854842
2       Total  39.0  7616.1792        NaN       NaN      NaN               NaN
📘 Saved ANOVA to: ccd_full_analysis_refined_with_plots/Compressive_ANOVA_manual.xlsx

--- Optimization of Predicted Strength ---

Predicted optimum for Compressive:
 → BFC (%): 0.0000
 → Temperature (°C): 200.0000
 → Duration (min): 40.0000
 → Predicted Strength (MPa): 48.9890
📘 Saved optimum parameters to: ccd_full_analysis_refined_with_plots/Compressive_OptimumParameters.xlsx

=== ANOVA and Optimization for Flexural ===

--- ANOVA (Manu

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>